In [6]:
import json
from os import makedirs, path
from copy import deepcopy

In [4]:
def clean_string(input_string, mode = None):
    """
    Utility function to clean the input string.
    The filters are chosen by hand when scraping the values.
    NOTE: Only add filters, don't remove them

    :param input_string: String to be cleaned
    :param mode: Useful to avoid adding spaces in case of select fields
    :return: Cleaned string
    """
    return input_string.strip().replace(u"\xa0â€‹", "").replace(u"â€‹", "").replace(u"\n", "").replace(u"\t", "").replace(u"\xa0", " " if (mode and mode == 'prof') else "").replace(u"\u200b", "").replace(u"\u00e0", "à")


def initialize_dataset():
    """
    Return an object to use for inizialization of a new dataset.

    :return: Initial dictionary for any dataset
    """
    return {
        "value": {
            "total": 0,
            "size": 0,
            "language": "en",
            "data": []
        }
    }


def append_data(dataset, to_append):
    """
    Utility function to mask away some code and make it more readable

    :param dataset: Dataset to append data to
    :param to_append: Data to append
    """
    dataset['value']['data'].append(to_append)


def save_dataset(dataset, name, file_format):
    """
    Save the dataset given in input

    :param dataset: Dataset to save
    :param name: Name of the dataset
    :param file_format: Format the dataset should be saved in
    """
    filename = f'../datasets/{name}.{file_format}'
    makedirs(path.dirname(filename), exist_ok=True)
    with open(filename, 'w', encoding='utf-8') as f:
        if file_format == 'json':
            json.dump(dataset, f, indent=2)


def set_total_size(dataset):
    """
    Sets the values of the 'total' and 'size' fields in a dictionary

    :param dataset: Dataset to manipulate
    """
    dataset['value']['total'] = len(dataset['value']['data'])
    dataset['value']['size'] = len(dataset['value']['data'])


In [15]:
with open('../datasets/original/person_en.json', 'r') as f:
    persons = json.load(f)

In [16]:
person_dataset = initialize_dataset()
for person in persons['value']['data']:
    person['id'] = person['identifier']
    del person['identifier']

    for position in person['position']:
        to_append = deepcopy(person)
        to_append['role'] = position['role']
        to_append['departmentId'] = position['unitId']
        to_append['phoneNumber'] = ""
        del to_append['position'], to_append['phone']

        append_data(person_dataset, to_append)
    
    if len(person['phone']) > 0:
        for phone in person['phone']:
            to_append = deepcopy(person)
            to_append['phoneNumber'] = phone
            to_append['role'] = ''
            to_append['departmentId'] = ''
            del to_append['position'], to_append['phone']

            append_data(person_dataset, to_append)

set_total_size(person_dataset)

save_dataset(person_dataset, 'generated/person_en_final', 'json')